In [4]:
import json
import numpy as np
from bokeh.plotting import figure
from bokeh.io import output_notebook, show
from bokeh.palettes import Category20_20
output_notebook()
colors = Category20_20

Loading BokehJS ...

In [5]:
info = json.loads(open('runs/1/info.json').read())
p = figure(title='CNN - MNIST', plot_width=600, plot_height=400)

c = 0
for k in info.keys():
    if k.startswith('uncertainty'):
        xy = np.array(info[k])
        area = np.trapz(y=xy[:, 1], x=xy[:, 0])
        p.line(xy[:, 0], xy[:, 1], legend='{0}: {1:.6f}'.format(k, area), color=colors[c], line_width=3)
        c += 1

p.xaxis.axis_label = 'data proportion'
p.yaxis.axis_label = 'accuracy'
p.legend.location = 'bottom_left'

show(p)

In [6]:
info = json.loads(open('runs/2/info.json').read())
p = figure(title='CNN - MNIST', plot_width=600, plot_height=400)

c = 0
for k in info.keys():
    if k.startswith('uncertainty'):
        xy = np.array(info[k])
        area = np.trapz(y=xy[:, 1], x=xy[:, 0])
        p.line(xy[:, 0], xy[:, 1], legend='{0}: {1:.6f}'.format(k, area), color=colors[c], line_width=3)
        c += 1

p.xaxis.axis_label = 'data proportion'
p.yaxis.axis_label = 'accuracy'
p.legend.location = 'bottom_left'

show(p)

In [7]:
info = json.loads(open('runs/3/info.json').read())
p = figure(title='MLP - Protein Structure', plot_width=800, plot_height=600)

c = 0
for k in info.keys():
    if k.startswith('uncertainty'):
        xy = np.array(info[k])
        area = np.trapz(y=xy[:, 1], x=xy[:, 0])
        p.line(xy[:, 0], xy[:, 1], legend='{0}: {1:.6f}'.format(k, area), color=colors[c], line_width=3)
        c += 1

p.xaxis.axis_label = 'data proportion'
p.yaxis.axis_label = 'RMSE'
p.legend.location = 'bottom_right'

show(p)

In [29]:
info = json.loads(open('runs/4/info.json').read())
#info['evaluations'][0].keys()
c = 0
p = figure(title='MLP - Protein Structure', plot_width=800, plot_height=600)
for k in info['evaluations'][0].keys():
    if not k.startswith('uncertainty') or k.endswith('auc'): continue
    xys = np.array([info['evaluations'][i][k] for i in range(len(info['evaluations']))])
    xs = np.mean(xys, axis=0)[:, 0]
    ys = np.mean(xys, axis=0)[:, 1]
    stds = np.std(xys, axis=0)[:, 1]
    print(k)

    # Bollinger shading glyph:
    band_x = np.append(xs, xs[::-1])
    band_y = np.append(ys - stds, (ys + stds)[::-1])
    p.patch(band_x, band_y, color=colors[c], fill_alpha=0.2)
    p.line(xs, ys, legend='{0}'.format(k), color=colors[c], line_width=3)
    c += 1

p.xaxis.axis_label = 'data proportion'
p.yaxis.axis_label = 'RMSE'
p.legend.location = 'bottom_right'
show(p)

uncertainty_classifer
uncertainty_nll_gaussian
uncertainty_nll_gaussian_std
uncertainty_predicted_std
uncertainty_predicted_std_det
uncertainty_std_predicted_mean
uncertainty_std_predicted_std
